In [10]:
%autoreload 2
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.nn import BCEWithLogitsLoss
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from datetime import date
import sys
sys.path.insert(0, '../utils')
from ground_truth_dataset import groundTruthDataset
from data_functions import splitDataset, returnLoaders
from metrics import returnPreReF
sys.path.insert(0, '../models')
from unet_model import UNet
sys.path.insert(0, '../train')
from training import train_one_epoch, valid_one_epoch 

In [ ]:
%load_ext autoreload

In [5]:
NUM_CLASSES = 2 # For ground truth data, there's 2 classes of Background, CAFO
batch_size = 8

In [12]:
dataset = groundTruthDataset("/datadrive/data/processed/planet_il_2019_daily_ms_truth")
datasets = splitDataset(dataset)
trainloader, validloader, testloader = returnLoaders(datasets, batch_size, True)
model = UNet(4, NUM_CLASSES) # 4 Channels, 2 Classes (background, CAFO)

# Training

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
train_num_batches = len(trainloader)
valid_num_batches = len(validloader)
train_num_examples = len(trainloader.dataset)
valid_num_examples = len(validloader.dataset)

In [15]:
#Set model to either cpu or gpu
model.to(device)            

#Define loss function
#Weight due to class imbalance
pos_weight = torch.tensor([1, 30]) #23 is good when doing 3 class
pos_weight = torch.reshape(pos_weight,(1,2,1,1)).to(device)
criterion = BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3,
                                     weight_decay = 1e-7)

In [16]:
today = date.today()
date_prefix = today.strftime("%m_%d")
log_dir_suffix = f"{date_prefix}_groundtruth_lr_{3e-4}_epochs_{10}_batch_size_{4}"
log_dir = "../logs/groundtruth/" + log_dir_suffix
writer = SummaryWriter(log_dir=log_dir)

In [17]:
torch.manual_seed(0)
np.random.seed(0)
model.zero_grad()
class_list = [0, 1]
for epoch in range(8):
    ### TRAINING ###
    print("Beginning Training in Epoch " + str(epoch))
    with tqdm(total = train_num_batches) as epoch_pbar:
        model.train()
        train_loss, train_correct, \
            train_IoU = train_one_epoch(epoch, train_num_batches, model, 
                                        device, trainloader, epoch_pbar, 
                                        optimizer, writer, criterion)

    ### VALIDATION ###
    print("Beginning Validation in Epoch " + str(epoch))
    valid_loss = []
    valid_correct = 0

    conf_matrix = np.zeros((2, 2))

    with tqdm(total = valid_num_batches) as epoch_pbar:
        model.eval()                           
        valid_loss, valid_correct, \
            conf_matrix, valid_IoU = valid_one_epoch(epoch, valid_num_batches, model, 
                                                     device, validloader, epoch_pbar, 
                                                     optimizer, writer, criterion,
                                                     conf_matrix, class_list)

Beginning Training in Epoch 0


RuntimeError: stack expects each tensor to be equal size, but got [3, 212, 217] at entry 0 and [3, 213, 216] at entry 1

In [ ]:
returnPreReF(conf_matrix, 1)

In [ ]:
# Testing it out on validation
val_example = next(iter(trainloader))
inputs = val_example[0].to(device)
labels = val_example[1].to(device)
with torch.no_grad():
    outputs = model(inputs)
_, predictions = torch.max(outputs, 1)
predictions_one_hot = torch.nn.functional.one_hot(predictions).permute(0, 3, 1, 2)
y_pred = predictions.flatten().cpu().numpy()
_, blah = torch.max(labels, 1)
y_true = blah.flatten().cpu().numpy()

In [ ]:
torch.sum((predictions[0] == 1).int())

In [ ]:
torch.save(model.state_dict(), f"../../../saved_models/finished/model8_10_ia_data.pth")